In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

from pathlib import Path

import numpy as np
import bottleneck as bn

from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
from astropy.table import Table, Column, vstack
from astropy.visualization import hist
from astropy import units as u
from astropy.time import Time

from ccdproc import combine, ImageFileCollection

import reproject

from photutils import DAOStarFinder, CircularAperture, CircularAnnulus, aperture_photometry, centroid_sources

from astrowidgets import ImageWidget

from glowing_waffles.photometry import detect_sources, faster_sigma_clip_stats, find_too_close, clipped_sky_per_pix_stats, add_to_photometry_table, photometry_on_directory

In [ ]:
directory_with_images = 'test_sub'
object_of_interest = 'kelt-16b'
sample_image_for_finding_stars = 'kelt-16-S001-R001-C100-r.fit'

# The name of your photometry file should include the night number (e.g. 2018-05-21), the object name 
# (e.g. kelt-16b) and the aperture radius (e.g. 12), so maybe: 2018-05-21-kelt-16b-aperture-12
name_of_output_file = 'replace_this' + '.fits'

## Pick aperture and annulus sizes

Probably the best way to do this for now is to upload one of the kelt-16 images to the notebook we used in class to plot star profiles. You want an aperture about 2×FWHM, an annulus with  an inner radius at least 10 pixels larger than that, and outer radius about 15 pixels larger than the inner.

In [ ]:
aperture_rad = 12
inner_annulus = aperture_rad + 15
outer_annulus = inner_annulus + 15

## Set some camera properties

In [ ]:
max_adu = 45000
gain = 1.47
read_noise = 9.42
dark_current = 0.014

## Read one image to use detect sources

This could be any image; I chose one roughly in the middle of the sequence. Ideally maybe we'd choose sources that were only in all of the images, but this will work for now.

In [ ]:
ccd = CCDData.read(str(Path(directory_with_images) / sample_image_for_finding_stars))

## Set pixels that are above point where CCD becomes non-linear to invalid value

This will ensure that the aperture sum is an invalid value if one or more of the pixels in the aperture is non-linear. *Change the value below to what you think is appropriate -- I'm going from memory.*

In [ ]:
ccd.data[ccd.data > max_adu] = np.nan

In [ ]:
iw = ImageWidget()
iw

In [ ]:
iw.load_fits(str(Path(directory_with_images) / sample_image_for_finding_stars))

In [ ]:
if object_of_interest:
    iw.center_on(SkyCoord.from_name(object_of_interest))

# Required stuff starts here

The background light from the sky needs to be removed before looking for sources. We'll do that by calculating the median. "Sigma clipping" is the term for excluding data that is far from the average...here we do it mostly to exclude the "bright" pixels (i.e. the stars) from our estimate of the background.

## Only include this if you have an image viewer above

It is handy to see where the detected sources are....

In [ ]:
stars = detect_sources(ccd, 8, 10)

In [ ]:
iw.reset_markers()
iw.marker = {'type': 'circle', 'color': 'lightgreen', 'radius': 10}
iw.add_markers(stars, x_colname='xcentroid', y_colname='ycentroid', pixel_coords_offset=0)

## Set up sky coordinates of target list

We'll want to the list of stars in RA/Dec so we can find them in each image. 

`star_locs` is waht is used later on to place the apertures

In [ ]:
star_locs = ccd.wcs.all_pix2world(stars['xcentroid'], stars['ycentroid'], 0)

### Check for stars closer than 2 × aperture radius and remove from list

In [ ]:
too_close = find_too_close(star_locs, aperture_rad)

In [ ]:
star_locs = (star_locs[0][~too_close], star_locs[1][~too_close])

### Include this only if you have the image viewer -- it shows which stars are being removed

In [ ]:
iw.marker = {'type': 'circle', 'color': 'red', 'radius': 20}
iw.add_markers(stars[too_close], x_colname='xcentroid', y_colname='ycentroid', pixel_coords_offset=0)

### Make up a unique ID for each star

In [ ]:
star_ids = np.arange(len(star_locs[0])) + 1

## We are ready to do photometry on all of the images

In [ ]:
all_phot = photometry_on_directory(directory_with_images, object_of_interest,
                                   star_locs, aperture_rad,
                                   inner_annulus, outer_annulus,
                                   max_adu, star_ids,
                                   gain, read_noise, dark_current)

### Write out the file!

In [ ]:
all_phot.write(name_of_output_file)